## Checks NDFD data for NaN values in data
---

In [1]:
import numpy as np
from datetime import datetime, timedelta
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import metpy
from pyproj import Proj

In [2]:
s_date = '20231115_1000'
e_date = '20231115_1630'

sdate = datetime.strptime(s_date, '%Y%m%d_%H%M')
edate = datetime.strptime(e_date, '%Y%m%d_%H%M')
halfhour = timedelta(minutes=30)

dates = np.arange(sdate, edate, halfhour)

In [3]:
for date_step in dates:

    ts = pd.to_datetime(date_step)
    YYYYMMDD_HHMM = ts.strftime('%Y%m%d_%H%M')

    File = "https://thredds.ucar.edu/thredds/dodsC/grib/NCEP/NDFD/NWS/CONUS/CONDUIT/NDFD_NWS_CONUS_conduit_2p5km_"+YYYYMMDD_HHMM+".grib2"

    ds = xr.open_dataset(File)

    ds = ds.metpy.parse_cf()
    ds = ds.metpy.assign_latitude_longitude(force=False)

    x, y = ds.x, ds.y

    #### Define function to get gridpoint nearest to a lat and lon

    def find_closest(array, value):
        idx = (np.abs(array-value)).argmin()
        return idx

    max_temp = ds.Maximum_temperature_height_above_ground_12_Hour_Maximum

    #### Define map projection & ETEC gridpoint

    proj_data = max_temp.metpy.cartopy_crs
    proj_data;

    pFull = Proj(proj_data)

    siteName = "ETEC"
    siteLat, siteLon = (42.68, -73.81) #lat & lon of gridpoint over ETEC
    siteX, siteY = pFull(siteLon, siteLat)
    siteXidx, siteYidx = find_closest(x, siteX), find_closest(y, siteY)

    #### Define max temp Variable

    forecastMax = max_temp.isel(x = siteXidx, y = siteYidx).isel()

    timeDimMax, vertDimMax = forecastMax.metpy.time.name, forecastMax.metpy.vertical.name

    idxTimeMax = slice(None, 2) # First time
    idxVertMax = 0 # First (and in this case, only) vertical level

    timeDictMax = {timeDimMax: idxTimeMax}
    vertDictMax = {vertDimMax: idxVertMax}

    forecastMax = forecastMax.isel(vertDictMax).isel(timeDictMax)

    #### Define wind speed variable

    windSpeed = ds.Wind_speed_height_above_ground
    windSpeed = windSpeed.isel(x = siteXidx, y = siteYidx).isel()

    timeDimWdsp, vertDimWdsp = windSpeed.metpy.time.name, windSpeed.metpy.vertical.name

    idxVertWdsp = 0 # First (and in this case, only) vertical level
    vertDictWdsp = {vertDimWdsp: idxVertWdsp}

    forecastWindSpeed = windSpeed.isel(vertDictWdsp)

    #### Define wind direction variable

    windDir = ds.Wind_direction_from_which_blowing_height_above_ground
    windDir = windDir.isel(x = siteXidx, y = siteYidx).isel()

    timeDimWdr, vertDimWdr = windDir.metpy.time.name, windDir.metpy.vertical.name

    idxVertWdr = 0
    vertDictWdr = {vertDimWdr: idxVertWdr}

    forecastWindDir = windDir.isel(vertDictWdr)

    #### Define weather string variable

    wx = ds.Weather_string_surface
    wx = wx.isel(x = siteXidx, y = siteYidx).isel()

    timeDimWx = wx.metpy.time.name

    #### Define min temp variable

    min_temp = ds.Minimum_temperature_height_above_ground_12_Hour_Minimum

    forecastMin = min_temp.isel(x = siteXidx, y = siteYidx).isel()

    timeDimMin, vertDimMin = forecastMin.metpy.time.name, forecastMin.metpy.vertical.name

    idxTimeMin = slice(None, 2) # First time
    idxVertMin = 0 # First (and in this case, only) vertical level

    timeDictMin = {timeDimMin: idxTimeMin}
    vertDictMin = {vertDimMin: idxVertMin}

    forecastMin = forecastMin.isel(vertDictMin).isel(timeDictMin)

    #### Define precip probability variable

    precip = ds.Total_precipitation_surface_12_Hour_Accumulation_probability_above_0p254

    forecastPrecip = precip.isel(x = siteXidx, y = siteYidx).isel()

    timeDimPrecip = forecastPrecip.metpy.time.name

    idxTimeFull = slice(None, 4) # First 4 times

    timeDictPrecip = {timeDimPrecip: idxTimeFull}

    forecastPrecip = forecastPrecip.isel(timeDictPrecip)

    #### Define cloud cover variable

    cloudCover = ds.Total_cloud_cover_surface

    forecastCloudCover = cloudCover.isel(x = siteXidx, y = siteYidx).isel()

    timeDimCloudCover = forecastCloudCover.metpy.time.name

    #### Define Temp variable

    temp = ds.Temperature_height_above_ground

    forecastTemp = temp.isel(x = siteXidx, y = siteYidx).isel()

    timeDimTemp, vertDimTemp = forecastTemp.metpy.time.name, forecastTemp.metpy.vertical.name

    idxVertTemp = 0

    vertDictTemp = {vertDimTemp: idxVertTemp}

    forecastTemp = forecastTemp.isel(vertDictTemp)

    #### Define dewpoint variable

    dewp = ds.Dewpoint_temperature_height_above_ground

    forecastDewp = dewp.isel(x = siteXidx, y = siteYidx).isel()

    timeDimDewp, vertDimDewp = forecastDewp.metpy.time.name, forecastDewp.metpy.vertical.name

    idxVertDewp = 0

    vertDictDewp = {vertDimDewp: idxVertDewp}

    forecastDewp = forecastDewp.isel(vertDictDewp)

    #### Define relative humidity variable

    rh = ds.Relative_humidity_height_above_ground

    forecastRH = rh.isel(x = siteXidx, y = siteYidx).isel()

    timeDimRH, vertDimRH = forecastRH.metpy.time.name, forecastRH.metpy.vertical.name

    idxVertRH = 0

    vertDictRH = {vertDimRH: idxVertRH}

    forecastRH = forecastRH.isel(vertDictRH)

    #### Define time lists at intervals of 1-hr & 12-hr

    times1 = forecastCloudCover.metpy.time.values

    times12 = forecastPrecip.metpy.time.values

    #### Define time lists for max and min temps

    timesMax = forecastMax.metpy.time.values

    timesMin = forecastMin.metpy.time.values
    
    timesrh = forecastRH.metpy.time.values

    #they get their own time list since they only exist for every other 12-hr period

    #### Check cloud cover for NaN

    i = 0

    cloudNanList=[]

    for time_step in times1:

        timeDictCloudCover = {timeDimCloudCover: i}
        forecastCloudCoverNew = forecastCloudCover.isel(timeDictCloudCover)

        nanCheck = forecastCloudCoverNew.isnull().values

        nanStr = str(nanCheck)

        cloudNanList.append(nanStr)

        i = i + 1

    cloudNanCount = cloudNanList.count('True')

    if cloudNanCount > 0:
        cloudNanCheck = True

    else:
        cloudNanCheck = False

    #### Check wind speed for NaN

    i = 0

    wdspNanList=[]

    for time_step in times1:

        timeDictWdsp = {timeDimWdsp: i}
        wdspNew = forecastWindSpeed.isel(timeDictWdsp)

        nanCheck = wdspNew.isnull().values

        nanStr = str(nanCheck)

        wdspNanList.append(nanStr)

        i = i + 1

    wdspNanCount = wdspNanList.count('True')

    if wdspNanCount > 0:
        wdspNanCheck = True

    else:
        wdspNanCheck = False

    #### Check wind direction for NaN

    i = 0

    wdrNanList=[]

    for time_step in times1:

        timeDictWdr = {timeDimWdr: i}
        wdrNew = forecastWindDir.isel(timeDictWdr)

        nanCheck = wdrNew.isnull().values

        nanStr = str(nanCheck)

        wdrNanList.append(nanStr)

        i = i + 1

    wdrNanCount = wdrNanList.count('True')

    if wdrNanCount > 0:
        wdrNanCheck = True

    else:
        wdrNanCheck = False

    #### Check weather string for NaN

    i = 0

    wxNanList=[]

    for time_step in times1:

        timeDictWx = {timeDimWx: i}
        wxNew = wx.isel(timeDictWx)

        nanCheck = wxNew.isnull().values

        nanStr = str(nanCheck)

        wxNanList.append(nanStr)

        i = i + 1

    wxNanCount = wxNanList.count('True')

    if wxNanCount > 0:
        wxNanCheck = True

    else:
        wxNanCheck = False

    #### Check dewpoint for NaN

    i = 0

    dewpNanList=[]

    for time_step in times1:

        timeDictDewp = {timeDimDewp: i}
        dewpNew = forecastDewp.isel(timeDictDewp)

        nanCheck = dewpNew.isnull().values

        nanStr = str(nanCheck)

        dewpNanList.append(nanStr)

        i = i + 1

    dewpNanCount = dewpNanList.count('True')

    if dewpNanCount > 0:
        dewpNanCheck = True

    else:
        dewpNanCheck = False

    #### Check temp for NaN

    i = 0

    tempNanList=[]

    for time_step in times1:

        timeDictTemp = {timeDimTemp: i}
        tempNew = forecastTemp.isel(timeDictTemp)

        nanCheck = tempNew.isnull().values

        nanStr = str(nanCheck)

        tempNanList.append(nanStr)

        i = i + 1

    tempNanCount = tempNanList.count('True')

    if tempNanCount > 0:
        tempNanCheck = True

    else:
        tempNanCheck = False

    #### Check max temp for NaN

    i = 0

    maxNanList=[]

    for time_step in timesMax:

        timeDictMax = {timeDimMax: i}
        maxNew = forecastMax.isel(timeDictMax)

        nanCheck = maxNew.isnull().values

        nanStr = str(nanCheck)

        maxNanList.append(nanStr)

        i = i + 1

    maxNanCount = maxNanList.count('True')

    if maxNanCount > 0:
        maxNanCheck = True

    else:
        maxNanCheck = False

    #### Check min temp for NaN

    i = 0

    minNanList=[]

    for time_step in timesMin:

        timeDictMin = {timeDimMin: i}
        minNew = forecastMin.isel(timeDictMin)

        nanCheck = minNew.isnull().values

        nanStr = str(nanCheck)

        minNanList.append(nanStr)

        i = i + 1

    minNanCount = minNanList.count('True')

    if minNanCount > 0:
        minNanCheck = True

    else:
        minNanCheck = False

    #### Check RH for NaN

    i = 0

    rhNanList=[]

    for time_step in timesrh:

        timeDictRH = {timeDimRH: i}
        rhNew = forecastRH.isel(timeDictRH)

        nanCheck = rhNew.isnull().values

        nanStr = str(nanCheck)

        rhNanList.append(nanStr)

        i = i + 1

    rhNanCount = rhNanList.count('True')

    if rhNanCount > 0:
        rhNanCheck = True

    else:
        rhNanCheck = False

    #### Display whether there is a NaN value or Not for each variable
    
    print('--------')
    
    print(YYYYMMDD_HHMM)

    print(cloudNanCheck)

    print(wdspNanCheck)

    print(wdrNanCheck)

    print(wxNanCheck)

    print(dewpNanCheck)

    print(tempNanCheck)

    print(maxNanCheck)

    print(minNanCheck)

    print(rhNanCheck)
    
print('--------')

--------
20231115_1000
False
False
False
False
False
False
False
False
False
--------
20231115_1030
False
False
False
False
False
False
False
False
False
--------
20231115_1100
False
False
False
False
False
False
False
False
False
--------
20231115_1130
False
False
False
False
False
False
False
False
False
--------
20231115_1200
False
False
False
False
False
False
False
False
False
--------
20231115_1230
False
False
False
False
False
False
False
False
False
--------
20231115_1300
False
False
False
False
False
False
False
False
False
--------
20231115_1330
False
False
False
False
False
False
False
False
False
--------
20231115_1400
False
False
False
False
False
False
False
False
False
--------
20231115_1430
False
False
False
False
False
False
False
False
False
--------
20231115_1500
False
False
False
False
False
False
False
False
False
--------
20231115_1530
False
False
False
False
False
False
False
False
False
--------
20231115_1600
False
False
False
False
False
False
False
False
False